# Tutorial: Compressor Unit Model with IAPWS Property Package



![](compressor_2.svg)

## Learning Outcomes

- Demonstrate use of the compressor unit model in IDAES
- Demonstrate different simulation options available

In this tutorial, we will simulate steam flow through a compressor unit model, using the IAPWS property package.
It is assumed that the turbine operates at steady state.
The inlet specifications are as follows:

* Flow Rate = 1 kmol/hr
* Mole fraction (H2O) = 1
* Pressure = 101325 Pa
* Temperature = 390 K 

We will simulate 2 different cases, depending on the operating specifications by the user:

**Case 1**: The compressor specifications are as follows:
   * Pressure Increase = 50000 Pa
   * Isentropic Efficiency = 0.9
   
**Case 2**: The compressor specifications are as follows:
   * Pressure Ratio = 1.49346
   
Compressor specification:
   * Isentropic Efficiency = 0.9

IDAES documentation reference for compressor model:https://idaes-pse.readthedocs.io/en/stable/

## Setting up the problem in IDAES

In the following cell, we will be importing the necessary components from Pyomo and IDAES.

In [1]:
# Import objects from pyomo package 
from pyomo.environ import ConcreteModel, SolverFactory, value

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False}) # dynamic or ss flowsheet needs to be specified here


# Import the IAPWS property package to create a properties block for the flowsheet
from idaes.generic_models.properties import iapws95
from idaes.generic_models.properties.helmholtz.helmholtz import (
    PhaseType
)

# Add properties parameter block to the flowsheet with specifications
m.fs.properties = iapws95.Iapws95ParameterBlock()

2020-08-06 18:24:05 [WARNING] idaes.core.property_meta: DEPRECATED: IDAES is moving to using Pyomo Units when defining default units, which are used to automatically determine units of measurement for quantities and convert where necessary. Users are strongly encouraged to convert their property packages to use Pyomo Units objects.


## Case 1:
Provide the turbine specifications (pressure change and efficieny) for the simulation.

### Add Turbine Unit

In [2]:
# Import compressor unit model from the model library
from idaes.generic_models.unit_models.pressure_changer import Compressor

# Create an instance of the compressor unit, attaching it to the flowsheet
# Specify that the property package to be used with the compressor is the one we created earlier.
m.fs.compr_case_1 = Compressor(default={"property_package": m.fs.properties})

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Call the degrees_of_freedom function, get intitial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

The initial DOF is 5


In [3]:
assert DOF_initial == 5

### Fix stream inlet conditions 

In [4]:
# Fix the stream inlet conditions
m.fs.compr_case_1.inlet.flow_mol[0].fix(1*1000/3600) # converting to mol/s as unit basis is mol/s

# Use htpx method to obtain the molar enthalpy of inlet stream at the given temperature and pressure conditions 
m.fs.compr_case_1.inlet.enth_mol[0].fix(iapws95.htpx(T=390, P=101325))
m.fs.compr_case_1.inlet.pressure[0].fix(101325)

2020-08-06 18:24:05 [WARNING] idaes.core.property_meta: DEPRECATED: IDAES is moving to using Pyomo Units when defining default units, which are used to automatically determine units of measurement for quantities and convert where necessary. Users are strongly encouraged to convert their property packages to use Pyomo Units objects.


### Fix compressor conditions

In [5]:
# Fix compressor conditions
m.fs.compr_case_1.deltaP.fix(50000)
m.fs.compr_case_1.efficiency_isentropic.fix(0.9)

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The final DOF is {0}".format(DOF_final))

The final DOF is 0


### Flowsheet Initialization

In [6]:
# Initialize the flowsheet, and set the output at WARNING
m.fs.compr_case_1.initialize(outlvl=idaeslog.WARNING)
# From the output it can be inferred that since there are no errors or warnings encountered during initialization, nothing is displayed 

### Obtaining Simulation Results

In [7]:
# Solve the simulation using ipopt
# Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory('ipopt')
solve_status = opt.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [8]:
from pyomo.opt import TerminationCondition, SolverStatus

# Check if termination condition is optimal
assert solve_status.solver.termination_condition == TerminationCondition.optimal
assert solve_status.solver.status == SolverStatus.ok

### View Results

In [9]:
# Display Outlet Pressure
m.fs.compr_case_1.outlet.pressure.display()

IndexedVar : Size=1, Index=fs.time
    Key : Lower : Value    : Upper        : Fixed : Stale : Domain
    0.0 :   0.1 : 151325.0 : 1000000000.0 : False : False : PositiveReals


In [10]:
# Display a readable report
m.fs.compr_case_1.report()


Unit : fs.compr_case_1                                                     Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                   : Value   : Fixed : Bounds
    Isentropic Efficiency : 0.90000 :  True : (None, None)
          Mechanical Work :  416.16 : False : (None, None)
          Pressure Change :  50000. :  True : (None, None)
           Pressure Ratio :  1.4935 : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                  Inlet     Outlet  
    Molar Flow (mol/s)            0.27778    0.27778
    Mass Flow (kg/s)            0.0050042  0.0050042
    T (K)                          390.00     433.30
    P (Pa)                     1.0132e+05 1.5132e+05
    Vapor Fraction                 1.0000     1.0000
    Molar Enthalpy (J/mol) Vap     48824.     50322.
    Molar Enthalpy (J/mol) 

In [11]:
import pytest

# Check results
assert m.fs.compr_case_1.outlet.pressure[0].value == pytest.approx(151325, abs=1e-2)

## Case 2:
Provide the desired compressor pressure ratio and efficiency for the simulation.

### Add Compressor Unit

In [12]:
# Create an instance of another compressor unit, attaching it to the flowsheet
# Specify that the property package to be used with the turbine is the one we created earlier.
m.fs.compr_case_2 = Compressor(default={"property_package": m.fs.properties})

# Call the degrees_of_freedom function, get intitial DOF
DOF_initial = degrees_of_freedom(m.fs.compr_case_2)
print("The initial DOF is {0}".format(DOF_initial))

The initial DOF is 5


In [13]:
assert DOF_initial == 5

### Fix stream inlet conditions 

In [14]:
# Fix the stream inlet conditions
m.fs.compr_case_2.inlet.flow_mol[0].fix(1*1000/3600) # converting to mol/s as unit basis is mol/s

# Use htpx method to obtain the molar enthalpy of inlet stream at the given temperature and pressure conditions 
m.fs.compr_case_2.inlet.enth_mol[0].fix(iapws95.htpx(T=390, P=101325))
m.fs.compr_case_2.inlet.pressure[0].fix(101325)

2020-08-06 18:24:06 [WARNING] idaes.core.property_meta: DEPRECATED: IDAES is moving to using Pyomo Units when defining default units, which are used to automatically determine units of measurement for quantities and convert where necessary. Users are strongly encouraged to convert their property packages to use Pyomo Units objects.


### Fix compressor pressure ratio & efficiency

In [15]:
# Fix compressor pressure ratio
m.fs.compr_case_2.ratioP.fix(1.49346)

# Fix compressor efficiency
m.fs.compr_case_2.efficiency_isentropic.fix(0.9)

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m.fs.compr_case_2)
print("The final DOF is {0}".format(DOF_final))

The final DOF is 0


In [16]:
assert DOF_final == 0

### Obtaining Simulation Results

In [17]:
# Solve the simulation using ipopt
# Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory('ipopt')
solve_status = opt.solve(m.fs.compr_case_2, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [18]:
# Check if termination condition is optimal
assert solve_status.solver.termination_condition == TerminationCondition.optimal
assert solve_status.solver.status == SolverStatus.ok

### View Results

In [19]:
# Display compressor pressure increase
m.fs.compr_case_2.outlet.pressure[0].display()

pressure : Pressure [Pa]
    Size=1, Index=None
    Key  : Lower : Value       : Upper        : Fixed : Stale : Domain
    None :   0.1 : 151324.8345 : 1000000000.0 : False : False : PositiveReals


In [20]:
# Display a readable report
m.fs.compr_case_2.report()


Unit : fs.compr_case_2                                                     Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                   : Value   : Fixed : Bounds
    Isentropic Efficiency : 0.90000 :  True : (None, None)
          Mechanical Work :  416.16 : False : (None, None)
          Pressure Change :  50000. : False : (None, None)
           Pressure Ratio :  1.4935 :  True : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                  Inlet     Outlet  
    Molar Flow (mol/s)            0.27778    0.27778
    Mass Flow (kg/s)            0.0050042  0.0050042
    T (K)                          390.00     433.30
    P (Pa)                     1.0132e+05 1.5132e+05
    Vapor Fraction                 1.0000     1.0000
    Molar Enthalpy (J/mol) Vap     48824.     50322.
    Molar Enthalpy (J/mol) 

In [21]:
import pytest

# Check results
assert m.fs.compr_case_2.outlet.pressure[0].value == pytest.approx(151324.8325, abs=1e-2)